### Do necessary imports

In [ ]:
from pyspark.sql import DataFrame, Row, SparkSession
from pyspark.sql.functions import substring, expr
import pandas as pd

# Temporary imports to load avro files
from os import listdir
from os.path import isfile, join

# Start spark session
sqlc = SparkSession \
    .builder \
    .appName("Master Thesis Analysis") \
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.2.1") \
    .getOrCreate()

# Constants
FILE_PATH = "openintel/"

### Load AVRO files DNS RR

In [ ]:
# TODO: connect to spark cluster, now manually load 1 avro folder for testing
my_dnsrr_df = sqlc.read.format("avro").load([join(FILE_PATH, f) for f in listdir(FILE_PATH) if isfile(join(FILE_PATH, f))])

# Load the known domain names
with open('CT-logs domeinen.txt', 'r') as f:
    known_names = [line.strip() for line in f]

with open('CT-logs domeinen.txt', 'r') as f:
    known_names_with_dot = [line.strip() + '.' for line in f]    

# TODO: delete this, manual overlap added for testing CT
known_names.append("18dbf4a18e441d4e8d27b9dacdcb61348453.postgres.database.azure.com")

# Transform known domain names to dataframes
my_known_names_df = sqlc.createDataFrame(pd.DataFrame({"domain": known_names }))
my_known_names_with_dot_df = sqlc.createDataFrame(pd.DataFrame({"query_name": known_names_with_dot }))

# Load the Resource Records into dataframes
with open('CAA records.txt', 'r') as f:
    caa_known = [line.split()[-1] for line in f]
    caa_known_df = sqlc.createDataFrame(pd.DataFrame({"caa_value": caa_known }))

with open('A IPv4 records.txt', 'r') as f:
    ipv4_known = [line.strip() for line in f]
    ipv4_known_df = sqlc.createDataFrame(pd.DataFrame({"ip4_address": ipv4_known }))
    
with open('MX records.txt', 'r') as f:
    mx_known = [line.split()[-1] for line in f]
    mx_known_df = sqlc.createDataFrame(pd.DataFrame({"mx_address": mx_known }))
    
with open('NS records.txt', 'r') as f:
    ns_known = [line.strip() for line in f]    
    ns_known_df = sqlc.createDataFrame(pd.DataFrame({"ns_address": ns_known }))

### OpenINTEL analysis

In [ ]:
# Select the data needed for every comparison
caa_data = my_dnsrr_df.where(my_dnsrr_df.response_type == "CAA").select('query_name', 'caa_value')
ipv4_data = my_dnsrr_df.where(my_dnsrr_df.response_type == "A").select('query_name', 'ip4_address')
mx_data = my_dnsrr_df.where(my_dnsrr_df.response_type == "MX").select('query_name', 'mx_address')
ns_data = my_dnsrr_df.where(my_dnsrr_df.response_type == "NS").select('query_name', 'ns_address')

# Obtain results per type
# First join on recognized value. Select only query name and only distinct values
# Then anti join with known names to find new domains
# Finally remove the trailing dot, since we need PQDNs for CT analysis
# TODO: probably remove CAA
caa_result = caa_data.join(caa_known_df, caa_data.caa_value == caa_known_df.caa_value, 'inner').select('query_name').distinct()
caa_result = caa_result.join(my_known_names_with_dot_df, 'query_name', how='left_anti')
caa_result = caa_result.withColumn("domain", expr("substring(query_name, 1, length(query_name)-1)")).select("domain")
caa_result.show(truncate=False)

ipv4_result = ipv4_data.join(ipv4_known_df, ipv4_data.ip4_address == ipv4_known_df.ip4_address, 'inner').select('query_name').distinct()
ipv4_result = ipv4_result.join(my_known_names_with_dot_df, 'query_name', how='left_anti')
ipv4_result = ipv4_result.withColumn("domain", expr("substring(query_name, 1, length(query_name)-1)")).select("domain")
ipv4_result.show(truncate=False)

mx_result = mx_data.join(mx_known_df, mx_data.mx_address == mx_known_df.mx_address, 'inner').select('query_name').distinct()
mx_result = mx_result.join(my_known_names_with_dot_df, 'query_name', how='left_anti')
mx_result = mx_result.withColumn("domain", expr("substring(query_name, 1, length(query_name)-1)")).select("domain")
mx_result.show(truncate=False)

ns_result = ns_data.join(ns_known_df, ns_data.ns_address == ns_known_df.ns_address, 'inner').select('query_name').distinct()
ns_result = ns_result.join(my_known_names_with_dot_df, 'query_name', how='left_anti')
ns_result = ns_result.withColumn("domain", expr("substring(query_name, 1, length(query_name)-1)")).select("domain")
ns_result.show(truncate=False)

# TODO: store intermediate results here for analysis




In [ ]:
# Join results with known domains.
# Currently excluding CAA, because it appeared to be used by non-government domains as well.
my_known_names_df = my_known_names_df.union(ipv4_result)
my_known_names_df = my_known_names_df.union(mx_result)
my_known_names_df = my_known_names_df.union(ns_result)

### Load AVRO files CT

In [ ]:
# TODO: connect to spark cluster, now manually load 1 avro file for testing
my_ct_df = sqlc.read.format("avro").load("Google-Rocketeer.1117550000-1117574999.avro")

### CT analysis

In [ ]:
# Filter the CT dataset on the known domain names
result = my_ct_df.join(my_known_names_df, my_ct_df.leaf_cert.subject.CN == my_known_names_df.domain, "inner")

In [ ]:
# Get the SAN off the certificates
sans = result.select(result.leaf_cert.all_domains).collect()

# Unique result set
uni_results = set()

# Loop rows
for row in sans:
    # Loop all_domain lists within row
    for domain in row.__getitem__('leaf_cert.all_domains'):
        # Add a domain to our set if it is not a wildcard domain + is not already in our known set.
        # Since we use a set, we do not need to check for duplicates
        if not domain.startswith('*') and domain not in known_names:
            uni_results.add(domain)

print(uni_results)

In [ ]:
# TODO: Store results

In [4]:
# Join results with known domains.
# Currently excluding CAA, because it appeared to be used by non-government domains as well.
my_known_names_df = my_known_names_df.union(ipv4_result)
my_known_names_df = my_known_names_df.union(mx_result)
my_known_names_df = my_known_names_df.union(ns_result)

### Load AVRO files CT

In [5]:
# TODO: connect to spark cluster, now manually load 1 avro file for testing
my_ct_df = sqlc.read.format("avro").load("Google-Rocketeer.1117550000-1117574999.avro")

### CT analysis

In [6]:
# Filter the CT dataset on the known domain names
result = my_ct_df.join(my_known_names_df, my_ct_df.leaf_cert.subject.CN == my_known_names_df.domain, "inner")

In [ ]:
# Get the SAN off the certificates
sans = result.select(result.leaf_cert.all_domains).collect()

# Unique result set
uni_results = set()

# Loop rows
for row in sans:
    # Loop all_domain lists within row
    for domain in row.__getitem__('leaf_cert.all_domains'):
        # Add a domain to our set if it is not a wildcard domain + is not already in our known set.
        # Since we use a set, we do not need to check for duplicates
        if not domain.startswith('*') and domain not in known_names:
            uni_results.add(domain)

print(uni_results)

In [ ]:
# TODO: Store results